# Rossmann

Please manually download the competition data from https://www.kaggle.com/c/rossmann-store-sales/data to ./data/raw

In [ ]:
%matplotlib inline
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from guacml import GuacMl

In [ ]:
stores = pd.read_csv('./data/raw/store.csv.zip')
train = pd.read_csv('./data/raw/train.csv.zip')
test = pd.read_csv('./data/raw/test.csv.zip')

In [ ]:
merged = train.merge(stores, how='outer', on=['Store'])
rand_stores = merged.Store.drop_duplicates().sample(10)
sample = merged[merged.Store.isin(rand_stores)]
sample = sample[sample.Sales > 0]

In [ ]:
guac = GuacMl(sample, 'Sales', 
              eval_metric='rmspe',
              target_transform='log',
              exclude_cols=['Customers'])

In [ ]:
guac.info()

In [ ]:
guac.make_time_series(date_split_col='Date', series_key_cols='Store', prediction_length=14, n_offset_models=4)
guac.plots.target_plot()

In [ ]:
guac.plots.tree()

In [ ]:
from guacml.preprocessing.feature_generation.historical_medians import HistoricalMedians

week_day_medians = HistoricalMedians([1, 5, 20], guac.config, logger=guac.logger, group_keys=['DayOfWeek', 'Promo'])
#guac.tree.insert_step_after('week_day_medians' ,'historical_medians', week_day_medians)
guac.plots.tree()

In [ ]:
guac.run(10)

In [ ]:
guac.plots.predictions_vs_actual('xg_boost')

In [ ]:
guac.model_results['xg_boost'].holdout_data